# 🤔
# <p style="color: #00BFFF;">**Ćwiczenia 10** </p>  
# <p style="color: #00BFFF">*Pre-trained word embeddings* </p>

Embeddingi to wektory reprezentujące słowa w przestrzeni wielowymiarowej. 

Są używane do przekształcania tekstu w formę zrozumiałą dla modeli maszynowego uczenia. 

Jest dużo gotowych przetrenowanych już zestawów word embeddings, które można użyć.

Popularne modele embeddingów to:


 * **googlowskie  Word2vec**  używa prediction model dla przewidzenia następnego słowa w zdaniu.
 
    Uczy się na podstawie kontekstu słów.

* **stanfordzkie GloVe**  używa count-based approach ( tak jak w macierzy word-word) i redukuje liczbę wymiarów.

    Uczy się na podstawie globalnych statystyk słów.

* **BERT** (Bidirectional Encoder Representations from Transformers) uczący się na podstawie kontekstu całych zdań.


 

Sposób przedstawienia znaczenia słowa w postaci wektorowej (tablica liczb), daje możliwość wykonania różnych "działań" na słowach. 

Klasycznym przykładem jest: 

Paryż-Francja+Włochy=Rzym



## <p style="color: orange;">Zadanie 1 </p>  🐸
Sprawdżmy czy NLP działa.

*( tak naprawde sprawdzimy tylko czy działa "matematyka" na słowach, ale możliwość traktowania słów jak wektorów jest punktem wyjścia do wszystkich ( albo prawie wszystkich) zadań przetwarzania języka naturalnego)*

Spróbujemy wymodelować działanie na wektorach i sprawdzić do jakiego wektora wynik jest najbardziej podobny w sensie podobieństwa znaczenia słowa. ( podobieństwo wektorów wystarczy, sprawdziliśmy już wczesniej że to działa nawet dla najprostszej miary cosinusowej)


Użyjemy Glove, korpus 6B.50d, z poniższego linka (pośrednio, ściągamy zipa i zostawiamy tylko jeden plik, ważne w jakim katalogu żeby dobrze się zlinkowało).
Zostawiamy najmniejszy plik, żebyśmy zdążyli policzyć. (można znaleźć link do pojedynczego zbiory glove.6B.50d.txt, wtedy nie ma potrzeby kasowania)


To jest 50 wymiarowe embeddings, co oznacza że każdy wektor ma 50 współrzędnych.

Stanford Glove website: 

https://nlp.stanford.edu/projects/glove/.



glove.6B.50d.txt


In [2]:
import pandas as pd
import csv

path = r"models/glove.6B.50d.txt"
words = pd.read_table(path, sep=" ", index_col=0, header=None, quoting=csv.QUOTE_NONE)

Otrzymamy 50 wymiarowe wektory z 400k słów.

Możemy obejrzeć jeden z nich, na przykład wektor

*Paris*

**Wyświetl wektor Paris** i wyjaśnij znaczenie współrzędnych tego wektora

In [3]:
# kod 
words[words.index == "paris"]

,1,2,3,4,5,6,7,8,9,10,...,41,42,43,44,45,46,47,48,49,50
0,,,,,,,,,,,,,,,,,,,,,
paris,0.76989,1.181,-1.1299,-0.74725,-0.5969,-1.0518,-0.46552,0.27009,-0.99243,-0.04864,...,0.84359,-0.25875,1.5616,-1.1199,0.091676,0.076675,-0.45084,-0.86104,0.97599,-0.35615


Teraz znajdźmy embedding dla słów, które się w tym zbiorze znajdują:

*France, Italy, Paris, Rome*


można użyć np. *word loc*, ale...

 nie trzeba; można stworzyć macierz dowolna metodą

Wyświetl macierz która powstanie, oraz macierz po normalizacji

In [4]:
#miejsce na kod macierzy coocurence, (znormalizowanej również)
import numpy as np

selected_words = ["france", "italy", "paris", "rome"]

embeddings = words.loc[selected_words]

coocurence_matrix = np.dot(embeddings, embeddings.T)
coocurence_df = pd.DataFrame(coocurence_matrix, index=selected_words, columns=selected_words)

In [5]:
coocurence_df

,france,italy,paris,rome
france,35.617476,26.398469,26.571788,17.723145
italy,26.398469,32.253093,18.623832,20.894403
paris,26.571788,18.623832,30.778870,19.298986
rome,17.723145,20.894403,19.298986,24.008865


In [6]:
row_sums = coocurence_matrix.sum(axis=1, keepdims=True)
normalized_matrix = coocurence_matrix / row_sums
normalized_df = pd.DataFrame(normalized_matrix, index=selected_words, columns=selected_words)

In [7]:
normalized_df

,france,italy,paris,rome
france,0.335031,0.248314,0.249944,0.166711
italy,0.268906,0.328544,0.189710,0.212839
paris,0.278900,0.195478,0.323058,0.202564
rome,0.216333,0.255042,0.235568,0.293058


No i sprawdżmy czy matematyka na słowach zadziała

czyli

*Paryż-Francja+Włochy=Rzym*

Stwórz wektor Paris-France+Italy



In [8]:
# miejsce na kod
left = words.loc["paris"].values - words.loc["france"] + words.loc["italy"]
right = words.loc["rome"]
(left == right).sum()

0

Zbadaj podobieństwo pomiędzy stworzonym wektorem a innymi słowami wyświetlając 10 najbardziej podobnych do niego słów 

( np.  używając find_closest_word, ale można też uzyć innej metody liczenia podobieństwa, szukaj metod similarity, 

wybierając metode pamiętaj, że poruszamy się wyważając korzyści między jakością wyniku a czasem obliczeń)



In [9]:
np.dot(words, right.T)

array([12.87599809,  9.74306152, 10.58793184, ..., -8.42813627,
       -3.25354317, -3.25354317])

In [10]:
a = np.dot(words, right.T)
b = pd.DataFrame(a, index=words.index, columns=["distance"])

In [11]:
# kod 
def find_closest_words(words, embedding, top_n=10):
    distances = np.dot(words, embedding.T)
    coocurence_df = pd.DataFrame(distances, index=words.index, columns=["distance"]).sort_values(by=["distance"], ascending=False)
    return coocurence_df.iloc[0:top_n, :]

find_closest_words(words, right, 10)

,distance
0,
rome,24.008865
italy,20.894403
cathedral,20.577377
milan,20.175303
sicherle,20.092326
pope,20.001254
madrid,19.379516
paris,19.298986
roman,19.273273


Czy najbardziej podobny do tego  wektora Paris-France+Italy jest wektor Rome ?

Czy to oznacza że taką relacje można powtórzyć dla innych zestawów słów?

## <p style="color: yellow;">Zadanie 2 </p> 🐳


 Powtórz działania dla poniższych zestawów :

**water-wet+fire= flames**


**winter-cold+summer=warm**


oraz klasycznego:

**king-men+woman=queen**

In [12]:
# kod zadania 2
def compare_vectors(index_1, index_2, index_3, index_4, words, top_n=10):
    left = words.loc[index_1] - words.loc[index_2] + words.loc[index_3]
    right = words.loc[index_4]
    
    print(f"Comparison: {(left==right).sum()}")
    
    coocurence_df = find_closest_words(words, right, 10)
    print(coocurence_df)

In [13]:
compare_vectors("water", "wet", "fire", "flames", words)

Comparison: 0
            distance
0                   
flames     28.240650
nahn       22.579730
debris     20.848968
billowed   20.623328
smoke      20.148238
tear       19.056865
billowing  18.841191
blaze      18.731457
avalanche  18.456331
debehogne  18.378496


In [14]:
compare_vectors("winter", "cold", "summer", "warm", words)

Comparison: 0
                 distance
0                        
warm            29.908686
1-800-535-4425  28.739116
moist           24.262903
dry             23.730344
temperatures    22.507354
drizzle         22.437515
warmer          22.102078
humid           22.089390
rain            21.985990
cold            21.807384


In [15]:
compare_vectors("king", "men", "women", "queen", words)

Comparison: 0
           distance
0                  
queen     26.964749
princess  25.256698
king      21.877506
throne    21.784407
royal     21.776677
daughter  21.687536
her       21.545704
empress   20.825105
prince    20.645073
mother    20.501926


Oprócz GloVe, istnieje kilka innych pretrenowanych embeddingów, które mogą być użyte do klasycznego zadania analogii, takiego jak 

"król - mężczyzna + kobieta = królowa" (Word2Vec i FasTex powinny działać również po polsku). 

## <p style="color: #6495ED;">Zadanie 3 </p> 🌍
Pobierz inne pretrenowane modele (np. Word2Vec, FastText, BERT)
<p style="color: red;">[dla chetnych] </p>  ELMo (Embeddings from Language Models) 

i dla każdego z nich powtórz zestaw angielskich analogii z poprzednich zadań.

Następnie spróbuj dodać polskie:

* **Warszawa - Polska + Niemcy = Berlin** 
* **Król - mężczyzna + kobieta = królowa** 
* **Paryż - Francja + Włochy = Rzym**
* **Siostra - kobieta + mężczyzna = brat** 

Porównaj jakość działania róznych modeli, również dla polskich zestawów. 
Możesz spróbować poprawić wyniki znajdując polskie modele.

Biblioteki:

* pip install gensim (word2Vec, FastTex)
* pip install transformers torch (BERT)
* <p style="color: red;">[dla chetnych] </p>  pip install allennlp (ElMo)


Modele:
* word2vec-google-news-300
* fasttext-wiki-news-subwords-300
* bert-base-uncased
* <p style="color: red;">[dla chetnych] </p>   Konfiguracja ELMo:

    options_file = "https://allennlp.s3.amazonaws.com/models/elmo/2x4096x512_2048cnn_2xhighway/elmo_2x4096x512_2048cnn_2xhighway_options.json"

    weight_file = "https://allennlp.s3.amazonaws.com/models/elmo/2x4096x512_2048cnn_2xhighway/elmo_2x4096x512_2048cnn_2xhighway_weights.hdf5"


In [16]:
# kod zadania 3
from gensim.models import KeyedVectors

fasttext_model = KeyedVectors.load(r'models/fasttext-wiki-news-subwords-300.model')

In [17]:
def check_similarity_f(model, words):
    left = model[words[0]] - model[words[1]] + model[words[2]]
    right = model[words[3]]

    print(f"Comparison: {(left == right).sum()}")
    print(f"Similar: {model.most_similar(left)}")

In [18]:
check_similarity_f(fasttext_model, ["king", "men", "women", "queen"])
check_similarity_f(fasttext_model, ["Paris", "France", "Italy", "Rome"])
check_similarity_f(fasttext_model, ["Warsaw", "Poland", "Germany", "Berlin"])
check_similarity_f(fasttext_model, ["sister", "women", "men", "brother"])

Comparison: 0
Similar: [('king', 0.8453638553619385), ('queen', 0.71834397315979), ('king-', 0.678547203540802), ('monarch', 0.6708823442459106), ('child-king', 0.6690906286239624), ('boy-king', 0.6576582789421082), ('king-making', 0.6454074382781982), ('kingship', 0.6315165758132935), ('princess', 0.6250572800636292), ('prince', 0.6195020079612732)]
Comparison: 0
Similar: [('Italy', 0.8525390028953552), ('Rome', 0.8008622527122498), ('Milan', 0.792957067489624), ('Bologna', 0.784528374671936), ('Naples', 0.7842305302619934), ('Italy-', 0.7604862451553345), ('Paris', 0.7551395297050476), ('Venice', 0.7510690093040466), ('Milano', 0.7439693808555603), ('Parioli', 0.7396515011787415)]
Comparison: 0
Similar: [('Berlin', 0.7403421401977539), ('Germany', 0.7287525534629822), ('Warsaw', 0.7193452715873718), ('Munich', 0.7039551734924316), ('Hamburg', 0.6924000978469849), ('Stuttgart', 0.6905930042266846), ('Frankfurt', 0.6903912425041199), ('Bonn', 0.6753316521644592), ('Cologne', 0.67145103

In [19]:
from transformers import BertTokenizer, BertModel
import torch.nn.functional as F
import torch

bert_tokenizer = BertTokenizer.from_pretrained(r"models/bert-base-uncased")
bert_model = BertModel.from_pretrained(r"models/bert-base-uncased")

In [20]:
def check_similarity_b(model, tokenizer, words):
    word_embeddings_matrix = model.embeddings.word_embeddings.weight
    
    ids_words = tokenizer(words, add_special_tokens=False)['input_ids']
    ids_words = [id_list[0] for id_list in ids_words]
    
    embedding_1 = word_embeddings_matrix[ids_words[0]] 
    embedding_2 = word_embeddings_matrix[ids_words[1]] 
    embedding_3 = word_embeddings_matrix[ids_words[2]] 
    embedding_4 = word_embeddings_matrix[ids_words[3]] 
    
    result_vector = embedding_1 - embedding_2 + embedding_3
    print(f"Comparison: {(result_vector == embedding_4).sum()}")
    
    result_vector_normalized = F.normalize(result_vector.unsqueeze(0), p=2, dim=1)
    word_embeddings_matrix_normalized = F.normalize(word_embeddings_matrix, p=2, dim=1)
    
    similarities = torch.matmul(result_vector_normalized, word_embeddings_matrix_normalized.transpose(0, 1))
    
    similarities = similarities.squeeze(0)
    
    sorted_indices = torch.argsort(similarities, descending=True)
    top_k = 5
    top_k_indices = sorted_indices[ : 3 + top_k]
    top_k_similarities = similarities[top_k_indices]
    top_k_words = tokenizer.convert_ids_to_tokens(sorted_indices)

    
    print(f"Similar:")
    for word, sim in zip(top_k_words, top_k_similarities):
        print(f"  {word}: {sim.item():.4f}")
    

In [21]:
check_similarity_b(bert_model, bert_tokenizer, ["king", "men", "women", "queen"])
check_similarity_b(bert_model, bert_tokenizer, ["Warsaw", "Poland", "Germany", "Berlin"])
check_similarity_b(bert_model, bert_tokenizer, ["Paris", "France", "Italy", "Rome"])
check_similarity_b(bert_model, bert_tokenizer, ["sister", "women", "men", "brother"])

Comparison: 0
Similar:
  king: 0.7536
  queen: 0.6097
  women: 0.4743
  kings: 0.4737
  queens: 0.4304
  prince: 0.4302
  princess: 0.4298
  ruler: 0.4134
Comparison: 0
Similar:
  warsaw: 0.7318
  germany: 0.6584
  berlin: 0.5645
  munich: 0.5633
  germans: 0.5404
  darmstadt: 0.5385
  vienna: 0.5370
  stuttgart: 0.5361
Comparison: 0
Similar:
  paris: 0.7137
  italy: 0.6913
  italian: 0.5560
  milan: 0.5440
  parisian: 0.5329
  italians: 0.5269
  naples: 0.5162
  rome: 0.5144
Comparison: 0
Similar:
  sister: 0.7488
  brother: 0.6498
  daughter: 0.4801
  men: 0.4801
  son: 0.4672
  father: 0.4654
  sisters: 0.4611
  niece: 0.4503


In [22]:
from gensim.models import KeyedVectors

word2vec_model = KeyedVectors.load(r"models/word2vec-google-news-300/word2vec-google-news-300.model")

In [25]:
def check_similarity(model, words):
    left = model[words[0]] - model[words[1]] + model[words[2]]
    right = model[words[3]]

    print(f"Comparison: {(left == right).sum()}")
    print(f"Similar: {model.most_similar(left)}")

In [26]:
check_similarity(word2vec_model, ["king", "men", "women", "queen"])
check_similarity(word2vec_model, ["Paris", "France", "Italy", "Rome"])
check_similarity(word2vec_model, ["Warsaw", "Poland", "Germany", "Berlin"])
check_similarity(word2vec_model, ["sister", "women", "men", "brother"])

Comparison: 1
Similar: [('king', 0.8527070879936218), ('queen', 0.6743921637535095), ('monarch', 0.6191632151603699), ('kings', 0.5753854513168335), ('crown_prince', 0.562209963798523), ('princess', 0.543317437171936), ('prince', 0.5246986150741577), ('sultan', 0.5236638784408569), ('ruler', 0.5165805220603943), ('monarchy', 0.5113592147827148)]
Comparison: 1
Similar: [('Milan', 0.7251641750335693), ('Rome', 0.7073156237602234), ('Italy', 0.684763491153717), ('Paris', 0.6693719625473022), ('Palermo_Sicily', 0.6014109253883362), ('Italian', 0.596384584903717), ('Tuscany', 0.5672276020050049), ('Sicily', 0.5654013752937317), ('Bologna', 0.5645887851715088), ('Bologna_Italy', 0.5513802170753479)]
Comparison: 1
Similar: [('Berlin', 0.7192326188087463), ('Warsaw', 0.7112792134284973), ('Frankfurt', 0.6706617474555969), ('Dusseldorf', 0.6601549386978149), ('Stuttgart', 0.6353572607040405), ('Munich', 0.6288971900939941), ('Germany', 0.6251928806304932), ('Vienna', 0.6169423460960388), ('Hamb

Które embeddingi lepiej odwzorowują zależności między słowami ?

## <p style="color: magenta;">Zadanie 4 </p> 🐊

Użyj pretrenowanych embeddingów do pokazania wieloznaczności słowa "bank". 

Zadanie polega na znalezieniu najbliższych angielskich słów do podanego niejednoznaczmego słowa i ustalenia kontekstu 
*<p style="color:yellow;">"bank"</p>* 

("bank" jako instytucja finansowa i "bank" jako brzeg rzeki)

Użyj modelu Word2Vec. 

Znajdź najbliższe (najbardziej podobne ) słowa ( 10 z najwyzszym podobieństwem) i na tej podstawie ustal kontekst w jakim słowo zostało zrozumiane.
Mozna spróbowac wymusić konkretny kontekst szukając podobieństw do "river_bank"



In [27]:
#kod zadania 4

word2vec_model.most_similar("bank"[:10])

[('banks', 0.7440759539604187),
 ('banking', 0.6901614665985107),
 ('Bank', 0.6698698401451111),
 ('lender', 0.634228527545929),
 ('banker', 0.6092953681945801),
 ('depositors', 0.6031531691551208),
 ('mortgage_lender', 0.5797975659370422),
 ('depositor', 0.5716428160667419),
 ('BofA', 0.5714625120162964),
 ('Citibank', 0.5589520931243896)]

## <p style="color: pink;">Zadanie 5 </p> 🦀

Powtórz zadanie 4 dla : 
 
  - *bat* (nietoperz vs kij do baseballa) 
  - *apple* (owoc vs firma technologiczna) 
  - *python* (język programowania vs wąż)

In [28]:
# kod zadania 5
word2vec_model.most_similar("apple"[:10])

[('apples', 0.720359742641449),
 ('pear', 0.6450697183609009),
 ('fruit', 0.6410146355628967),
 ('berry', 0.6302294731140137),
 ('pears', 0.6133960485458374),
 ('strawberry', 0.6058261394500732),
 ('peach', 0.6025872826576233),
 ('potato', 0.5960935354232788),
 ('grape', 0.5935865044593811),
 ('blueberry', 0.5866668224334717)]

In [29]:
word2vec_model.most_similar("bat"[:10])

[('bats', 0.7677518725395203),
 ('batting', 0.6346984505653381),
 ('Pinch_hitter_Brayan_Pena', 0.6011941432952881),
 ('batsman', 0.5579798817634583),
 ('batted', 0.5542199611663818),
 ('Hawaiian_hoary', 0.5447419881820679),
 ('Lelands.com_auctioned', 0.5397745370864868),
 ('yelled_Cheater', 0.5380043983459473),
 ('wicketkeeper_Andrew_Hodd', 0.5371009111404419),
 ('lefthanded_batter', 0.53566575050354)]

In [30]:
word2vec_model.most_similar("python"[:10])

[('pythons', 0.6688377857208252),
 ('Burmese_python', 0.6680365204811096),
 ('snake', 0.6606293320655823),
 ('crocodile', 0.6591362953186035),
 ('boa_constrictor', 0.6443519592285156),
 ('alligator', 0.6421657204627991),
 ('reptile', 0.6387745141983032),
 ('albino_python', 0.6158880591392517),
 ('croc', 0.6083583831787109),
 ('lizard', 0.6013416647911072)]

## <p style="color: #7FFFD4">Zadanie 6 </p> 🏟
Powtórz instrukcję zadania 4 dla polskich słów o różnych znaczeniach: 
  - *zamek* (budowla vs mechanizm zamykający), 
  - *piła* (narzędzie vs czasownik w przeszłości)
  - *list* (zapis wiadomości vs nachylenie statku; w kontekście żeglugi, "list" oznacza kąt nachylenia statku na boki (do portu lub do sterburty) w stanie równowagi, bez działania zewnętrznych sił


In [36]:
# kod zadania 6
fasttext_model.most_similar("zamek"[:10])

[('Zamek', 0.8126124739646912),
 ('zámek', 0.8014318943023682),
 ('Ogród', 0.7847594618797302),
 ('zamku', 0.7834646701812744),
 ('rynek', 0.7772862911224365),
 ('rzecz', 0.7753502130508423),
 ('Dziwnów', 0.7723685503005981),
 ('rzeczy', 0.7715327143669128),
 ('Książ', 0.7680349946022034),
 ('Podzamcze', 0.7647734880447388)]

In [39]:
fasttext_model.most_similar("pila"[:10])

[('pilau', 0.7341576218605042),
 ('pilav', 0.7166672348976135),
 ('pilum', 0.6975015997886658),
 ('palla', 0.6828243136405945),
 ('Mpila', 0.6811730861663818),
 ('pilaris', 0.6811076402664185),
 ('maro', 0.6795235872268677),
 ('bura', 0.6751182079315186),
 ('pilate', 0.6681479811668396),
 ('pilar', 0.6631638407707214)]

In [38]:
fasttext_model.most_similar("list"[:10])

[('lists', 0.84706711769104),
 ('listing', 0.7999898195266724),
 ('sub-list', 0.7774856686592102),
 ('listed', 0.7744861841201782),
 ('list-', 0.7545185089111328),
 ('lists-', 0.7412810325622559),
 ('non-list', 0.739302933216095),
 ('list--', 0.7241465449333191),
 ('list-table', 0.7234554886817932),
 ('listable', 0.7148085832595825)]

## 😎 Wnioski
* Który model działał najlepiej?


* Który model działał najszybciej?

* Który był najłatwiejszy w uzyciu?


* Który zadziałał dla polskiego?


